<a href="https://colab.research.google.com/github/Deep-Learning-Qatar/EEG-Vision/blob/main/Image_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image-Net

Reference: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [1]:
config = dict(
    model_nr = 1,

    # Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
    model_name = "inception",

    # Number of classes in the dataset
    num_classes = 40,

    # Batch size for training (change depending on how much memory you have)
    batch_size = 64,

    # Number of epochs to train for
    num_epochs = 15,

    # Flag for feature extracting. When False, we finetune the whole model,
    #   when True we only update the reshaped layer params
    feature_extract = True,

    lr = 0.001,
)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Imports

In [2]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
cuda = torch.cuda.is_available()
# Detect if we have a GPU available
device = torch.device("cuda:0" if cuda else "cpu")
print(cuda)

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101
True


In [3]:
!pip install wandb --upgrade
import wandb
wandb.login()

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.25)


wandb: Currently logged in as: sbaumann (use `wandb login --relogin` to force relogin)


True

# Model Initialization


In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):

    wandb.watch(model, criterion, log="all", log_freq=10)

    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            if phase == 'train':
                wandb.log({"epoch": epoch, "train_loss": epoch_loss, "train_acc": epoch_acc})
            elif phase == 'val':
                wandb.log({"epoch": epoch, "val_loss": epoch_loss, "val_acc": epoch_acc})
            else:
                print('Did not log')
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        
        print()

        # Save model every epoch
        filename = 'ImageNet' + str(config['model_nr']) + 'epoch' + str(epoch+1) + '.pth'
        torch.save(model.state_dict(), filename)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

Set Model Parameters’ .requires_grad attribute

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

Initialize and Reshape the Networks

In [6]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


# Initialize the model for this run
model_ft, input_size = initialize_model(config['model_name'], config['num_classes'], config['feature_extract'], use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth



Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), str

# Data Loading

In [7]:
# Load image data
data_by_image = torch.load('/content/gdrive/MyDrive/11-785 Deep Learning/Project/data_by_image.pth')
# Convert images back to PIL to be able to transform (crop)
data_by_image = {i: transforms.ToPILImage()(data_by_image[i]).convert("RGB") for i in data_by_image}

# Load EEG datasets
EEG_datasets = torch.load('/content/gdrive/MyDrive/11-785 Deep Learning/Project/EEG_datasets.pth')

In [8]:
# Create dataset for Images using EEG data
class ImageDataset(data.Dataset):
    def __init__(self, image_data, EEG_data, transform):
        self.image_data = image_data
        self.EEG_data = EEG_data
        self.image_ids = list(EEG_data.keys())
        self.transform = transform
        self.length = len(self.image_ids)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        image_id = self.image_ids[index]
        image = self.transform(self.image_data[image_id])
        label = self.EEG_data[image_id][0]['label']
        return image, label

In [9]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_image_dataset = ImageDataset(data_by_image, EEG_datasets['train'], data_transforms['train'])
train_image_loader_args = dict(shuffle=True, batch_size=config['batch_size'], num_workers=2) if cuda\
                    else dict(shuffle=True, batch_size=config['batch_size'])
train_image_loader = data.DataLoader(train_image_dataset, **train_image_loader_args)

val_image_dataset = ImageDataset(data_by_image, EEG_datasets['val'], data_transforms['val'])
val_image_loader_args = dict(shuffle=False, batch_size=config['batch_size'], num_workers=2) if cuda\
                    else dict(shuffle=False, batch_size=config['batch_size'])
val_image_loader = data.DataLoader(val_image_dataset, **val_image_loader_args)

test_image_dataset = ImageDataset(data_by_image, EEG_datasets['test'], data_transforms['val'])
test_image_loader_args = dict(shuffle=False, batch_size=config['batch_size'], num_workers=2) if cuda\
                    else dict(shuffle=False, batch_size=config['batch_size'])
test_image_loader = data.DataLoader(test_image_dataset, **test_image_loader_args)

dataloaders_dict = dict(
    train=train_image_loader,
    val=val_image_loader,
    test= test_image_loader,
)

In [10]:
del EEG_datasets
del data_by_image

# Optimizer and Criterion Creation

In [11]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if config['feature_extract']:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=config['lr'], momentum=0.9)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


# Run Training and Validation Step

In [ ]:
hyperparameters = config

In [ ]:
%cd /content/gdrive/MyDrive/11-785 Deep Learning/Project/Image-Net

with wandb.init(project='Image-Net', entity='eegvision', config=hyperparameters):
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    # Train and evaluate
    model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,
                                num_epochs=config.num_epochs, is_inception=(config.model_name=="inception"))

/content/gdrive/MyDrive/11-785 Deep Learning/Project/Image-Net


Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


train Loss: 3.2702 Acc: 0.2071
val Loss: 2.0515 Acc: 0.6030

Epoch 1/14
----------
train Loss: 1.8064 Acc: 0.6521
val Loss: 1.0849 Acc: 0.7940

Epoch 2/14
----------
train Loss: 1.2188 Acc: 0.7534
val Loss: 0.8315 Acc: 0.8241

Epoch 3/14
----------
train Loss: 1.0740 Acc: 0.7697
val Loss: 0.7182 Acc: 0.8392

Epoch 4/14
----------
train Loss: 0.9171 Acc: 0.7966
val Loss: 0.6457 Acc: 0.8543

Epoch 5/14
----------
train Loss: 0.8885 Acc: 0.7997
val Loss: 0.6002 Acc: 0.8593

Epoch 6/14
----------
train Loss: 0.7864 Acc: 0.8235
val Loss: 0.5635 Acc: 0.8693

Epoch 7/14
----------
train Loss: 0.7924 Acc: 0.8129
val Loss: 0.5386 Acc: 0.8643

Epoch 8/14
----------
train Loss: 0.7556 Acc: 0.8110
val Loss: 0.5166 Acc: 0.8794

Epoch 9/14
----------
train Loss: 0.6680 Acc: 0.8429
val Loss: 0.5035 Acc: 0.8693

Epoch 10/14
----------
train Loss: 0.6757 Acc: 0.8398
val Loss: 0.4843 Acc: 0.8844

Epoch 11/14
----------
train Loss: 0.7001 Acc: 0.8323
val Loss: 0.4710 Acc: 0.8844

Epoch 12/14
----------
t

epoch,14
train_loss,0.57462
train_acc,0.8567
_runtime,244
_timestamp,1618015043
_step,29
val_loss,0.45724
val_acc,0.88442


epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇██
train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁
train_acc,▁▆▇▇▇▇█▇▇██████
_runtime,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▆▆▇▇▇▇▇█▇█████


# Test Model

In [20]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0., 0
        predictions = []

        for feats, labels in test_loader:
            feats, labels = feats.to(device), labels.to(device)
            
            outputs = model(feats)

            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.append(predicted.cpu().numpy())
            
            del feats
            del labels

        acc = correct / total
        return predictions, acc

In [21]:
best_epoch = 10

%cd /content/gdrive/MyDrive/11-785 Deep Learning/Project/Image-Net
filename = 'ImageNet' + str(config['model_nr']) + 'epoch' + str(best_epoch) + '.pth'
model_ft.load_state_dict(torch.load(filename))

test_predictions, test_acc = test(model_ft, test_image_loader)
test_predictions = [item for sublist in test_predictions for item in sublist]
print(test_acc)

In [26]:
print(test_predictions[:5])
for i in range(5):
    print(test_image_dataset[i][1])

[27, 26, 16, 2, 39]
27
26
16
2
39
